In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
print(os.listdir('/content/gdrive/My Drive/face_mask-dataset'))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
['train', 'valid']


In [2]:
import tensorflow as tf
from keras import Model, Sequential, layers, models
from keras.applications import MobileNetV2
from keras.optimizers import Adam
from PIL import ImageFile
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
image_width = 224 
image_height = 224 
batch_size = 32
epochs = 20
initial_lr = 1e-4
train_path = '/content/gdrive/My Drive/face_mask-dataset/train'
valid_path = '/content/gdrive/My Drive/face_mask-dataset/valid'

In [4]:
base_model = MobileNetV2(weights='imagenet', include_top=False,input_tensor= layers.Input(shape=(224,224,3)))
head_model = base_model.output
head_model = layers.AveragePooling2D((7,7))(head_model)
head_model = layers.Flatten(name='Flatten')(head_model)
head_model = layers.Dense(128,activation='relu')(head_model)
head_model = layers.Dropout(0.5)(head_model)
head_model = layers.Dense(2, activation='softmax')(head_model)

model = Model(inputs=base_model.input, outputs= head_model)

for layer in base_model.layers:
  layer.trainable=False

opt = Adam(learning_rate=initial_lr, decay = initial_lr/epochs)
model.compile(loss='binary_crossentropy',optimizer= opt, metrics =['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 0s 0us/step


In [5]:
train_data_generator = ImageDataGenerator(
    horizontal_flip = True, 
    rescale = 1./255,
    rotation_range = 20,
    zoom_range=0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range=0.15,
    fill_mode = "nearest"
)

valid_data_generator = ImageDataGenerator(
    rescale = 1./255
)
train_generator = train_data_generator.flow_from_directory(
    train_path, 
    target_size = (image_width, image_width),
    class_mode = 'categorical'
)
valid_generator = valid_data_generator.flow_from_directory(
    valid_path,
    target_size = (image_width, image_width),
    class_mode = 'categorical'

)

Found 1101 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


In [6]:
model.fit_generator(train_generator, 
                    steps_per_epoch=552//batch_size,
                    validation_data = valid_generator,
                    validation_steps = 138//batch_size,
                    epochs = 20
                    )

Epoch 1/20
17/17 [==============================] - 173s 10s/step - loss: 0.7458 - accuracy: 0.5956 - val_loss: 0.4154 - val_accuracy: 0.8438
Epoch 2/20
17/17 [==============================] - 141s 8s/step - loss: 0.5169 - accuracy: 0.7543 - val_loss: 0.1763 - val_accuracy: 0.8984
Epoch 3/20
17/17 [==============================] - 51s 3s/step - loss: 0.4573 - accuracy: 0.7776 - val_loss: 0.2611 - val_accuracy: 0.9478
Epoch 4/20
17/17 [==============================] - 42s 2s/step - loss: 0.3659 - accuracy: 0.8438 - val_loss: 0.2317 - val_accuracy: 0.8984
Epoch 5/20
17/17 [==============================] - 49s 3s/step - loss: 0.3125 - accuracy: 0.8603 - val_loss: 0.2665 - val_accuracy: 0.9652
Epoch 6/20
17/17 [==============================] - 42s 2s/step - loss: 0.2916 - accuracy: 0.8800 - val_loss: 0.2724 - val_accuracy: 0.9219
Epoch 7/20
17/17 [==============================] - 48s 3s/step - loss: 0.2885 - accuracy: 0.8724 - val_loss: 0.1610 - val_accuracy: 0.9478
Epoch 8/20
17/17 

In [7]:
model.save('face_mask_detection.h5')